In [1]:
import json

import pandas as pd
from tqdm import tqdm
import numpy as np
import time
from scipy import spatial

In [2]:
chunksize = 10 ** 6

chunks = []

with pd.read_csv("passage_vectors_glove50d.csv", chunksize=chunksize) as reader:
    for chunk in reader:
        chunks.append(chunk)

In [3]:
documents = pd.DataFrame()
documents = documents.append(chunks)

<ipython-input-3-a550b6cb8543>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  documents = documents.append(chunks)


In [4]:
chunksize = 10 ** 6

chunks2 = []

with pd.read_csv("queries_vectors_glove50d.csv", chunksize=chunksize) as reader:
    for chunk in reader:
        chunks2.append(chunk)

In [5]:
queries = pd.DataFrame()
queries = queries.append(chunks2)

<ipython-input-5-b8b0a931e25a>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  queries = queries.append(chunks2)


In [6]:
import time
import ast
import re

In [7]:
documents['vector'] = documents['vector'].map(lambda vec: ast.literal_eval(','.join(re.sub(r'(?<=\d)(\s+)(?=-?\d)', ',', vec).splitlines())))

In [8]:
documents

,doc_id,passage,vector
0,0,the presence of communication amid scientific ...,"[0.42492138, 0.26413426, -0.05272629, -0.04136..."
1,1,the manhattan project and its atomic bomb help...,"[0.37462739, 0.26258059, 0.05782766, 0.0049764..."
2,2,essay on the manhattan project - the manhattan...,"[0.4856806, 0.28033044, 0.0116571, 0.01116388,..."
3,3,the manhattan project was the name for a proje...,"[0.427263244, 0.170399548, -0.0260498036, 0.03..."
4,4,versions of each volume as well as complementa...,"[0.30266934, 0.35406677, -0.0425143777, 0.2473..."
...,...,...,...
8841818,8841818,when metal salts emit short wavelengths of vis...,"[0.35505125, 0.41410419, 0.225286776, -0.09446..."
8841819,8841819,thousands of people across the united states w...,"[0.312043481, 0.308481201, -0.0970539805, 0.08..."
8841820,8841820,"the recipe that creates blue, for example, inc...","[0.294822429, 0.32258091, -0.0608234215, 0.187..."
8841821,8841821,"on independence days of yore, old-timey crowds...","[0.313339871, 0.181715464, 0.0369753316, 0.000..."


In [9]:
retrieved_docs = pd.read_csv('relevant_docs_for_100_queries_test.csv')

In [10]:
retrieved_docs

,qid,docids
0,156493,"[8182161, 6139386, 3288600, 3288596, 2259183, ..."
1,1110199,"[3838645, 554521, 398442, 4511137, 5218014, 81..."
2,1063750,"[4788295, 2997653, 4337527, 6093907, 2981566, ..."
3,130510,"[1494936, 7125239, 799647, 7501563, 996732, 99..."
4,489204,"[1479542, 1051498, 1950269, 2268657, 973923, 2..."
5,573724,"[2417985, 394136, 394139, 3365638, 7384325, 71..."
6,168216,"[3344828, 1381477, 3830857, 3174840, 1735360, ..."
7,1133167,"[5674622, 8160230, 8255705, 190809, 6467522, 8..."
8,527433,"[8617271, 6186391, 269423, 1251475, 4903731, 1..."
9,1037798,"[8760867, 8760864, 3641634, 8760873, 4788864, ..."


In [11]:
queries['vector'] = queries['vector'].map(lambda vec: ast.literal_eval(','.join(re.sub(r'(?<=\d)(\s+)(?=-?\d)', ',', vec).splitlines())))

In [12]:
retrieved_docs['docids'] = retrieved_docs['docids'].map(lambda vec: ast.literal_eval(','.join(re.sub(r'(?<=\d)(\s+)(?=-?\d)', ',', vec).splitlines())))

In [13]:
embeddings_1 = []

for passageIDs in tqdm(retrieved_docs['docids']):
    embedding = []
    for pID in passageIDs:
        embedding.append(documents[documents['doc_id']==pID].values[0][2])
    embeddings_1.append(embedding)

100%|██████████| 43/43 [09:14<00:00, 12.90s/it]


In [14]:
retrieved_docs['embeddings_docs'] = embeddings_1

In [15]:
retrieved_docs['embeddings_queries'] = queries['vector']

In [16]:
retrieved_docs

,qid,docids,embeddings_docs,embeddings_queries
0,156493,"[8182161, 6139386, 3288600, 3288596, 2259183, ...","[[0.388196574, 0.244249168, 0.137481624, -0.00...","[0.31343375, 0.511575, -0.4668265, -0.1621775,..."
1,1110199,"[3838645, 554521, 398442, 4511137, 5218014, 81...","[[0.42657155, 0.19579138, 0.28220812, 0.102177...","[0.16202775, 0.795145, -0.46033125, -0.23275, ..."
2,1063750,"[4788295, 2997653, 4337527, 6093907, 2981566, ...","[[0.355166434, 0.106690241, 0.14072722, -0.102...","[0.3308615, 0.711035, -0.1855625, -0.4082575, ..."
3,130510,"[1494936, 7125239, 799647, 7501563, 996732, 99...","[[0.377713008, -0.00231133514, -0.293942608, -...","[0.085094, 0.504474, -0.1749988, 0.2761902, 0...."
4,489204,"[1479542, 1051498, 1950269, 2268657, 973923, 2...","[[0.550560748, 0.171315066, -0.165249143, -0.1...","[0.1761, 0.2345, -0.37276, 0.03199, 0.33063, 0..."
5,573724,"[2417985, 394136, 394139, 3365638, 7384325, 71...","[[0.26325938, 0.46548753, 0.02265388, 0.192594...","[0.51936, 0.6926975, -0.31155, -0.0028925, 0.2..."
6,168216,"[3344828, 1381477, 3830857, 3174840, 1735360, ...","[[0.71077084, 0.12540752, 0.06708402, 0.216699...","[0.008874, 0.137065, -0.347945, 0.0185375, 0.2..."
7,1133167,"[5674622, 8160230, 8255705, 190809, 6467522, 8...","[[0.1223981, 0.49183291, -0.07783549, 0.525699...","[0.70916667, 0.31568333, -0.44523667, 0.076183..."
8,527433,"[8617271, 6186391, 269423, 1251475, 4903731, 1...","[[0.42529326, 0.29431083, -0.09624058, 0.23385...","[0.786148, 0.16785, -0.396498, -0.0035102, -0...."
9,1037798,"[8760867, 8760864, 3641634, 8760873, 4788864, ...","[[0.16464998, 0.24151766, 0.03492415, 0.126383...","[0.51637429, 0.44778393, -0.30746774, 0.142171..."


In [17]:
cosineDist = [[1- spatial.distance.cosine(q_embedding, p_embeddings) for p_embeddings in retrieved_docs['embeddings_docs'][0]] for q_embedding in retrieved_docs['embeddings_queries']]

In [18]:
retrieved_docs['cosineDist'] = cosineDist

In [19]:
retrieved_docs

,qid,docids,embeddings_docs,embeddings_queries,cosineDist
0,156493,"[8182161, 6139386, 3288600, 3288596, 2259183, ...","[[0.388196574, 0.244249168, 0.137481624, -0.00...","[0.31343375, 0.511575, -0.4668265, -0.1621775,...","[0.7706824998903609, 0.7534134139863174, 0.785..."
1,1110199,"[3838645, 554521, 398442, 4511137, 5218014, 81...","[[0.42657155, 0.19579138, 0.28220812, 0.102177...","[0.16202775, 0.795145, -0.46033125, -0.23275, ...","[0.7444827116295556, 0.7314614521048374, 0.769..."
2,1063750,"[4788295, 2997653, 4337527, 6093907, 2981566, ...","[[0.355166434, 0.106690241, 0.14072722, -0.102...","[0.3308615, 0.711035, -0.1855625, -0.4082575, ...","[0.6862237295040958, 0.6637043734807618, 0.679..."
3,130510,"[1494936, 7125239, 799647, 7501563, 996732, 99...","[[0.377713008, -0.00231133514, -0.293942608, -...","[0.085094, 0.504474, -0.1749988, 0.2761902, 0....","[0.6697042269158479, 0.6340808852957971, 0.664..."
4,489204,"[1479542, 1051498, 1950269, 2268657, 973923, 2...","[[0.550560748, 0.171315066, -0.165249143, -0.1...","[0.1761, 0.2345, -0.37276, 0.03199, 0.33063, 0...","[0.7521369989199707, 0.73420385978584, 0.76684..."
5,573724,"[2417985, 394136, 394139, 3365638, 7384325, 71...","[[0.26325938, 0.46548753, 0.02265388, 0.192594...","[0.51936, 0.6926975, -0.31155, -0.0028925, 0.2...","[0.7260829064947356, 0.7145796120067109, 0.734..."
6,168216,"[3344828, 1381477, 3830857, 3174840, 1735360, ...","[[0.71077084, 0.12540752, 0.06708402, 0.216699...","[0.008874, 0.137065, -0.347945, 0.0185375, 0.2...","[0.6786883758466433, 0.6661525455599271, 0.694..."
7,1133167,"[5674622, 8160230, 8255705, 190809, 6467522, 8...","[[0.1223981, 0.49183291, -0.07783549, 0.525699...","[0.70916667, 0.31568333, -0.44523667, 0.076183...","[0.7666739141391231, 0.7425141883136608, 0.776..."
8,527433,"[8617271, 6186391, 269423, 1251475, 4903731, 1...","[[0.42529326, 0.29431083, -0.09624058, 0.23385...","[0.786148, 0.16785, -0.396498, -0.0035102, -0....","[0.6564955120524906, 0.6290717790738949, 0.663..."
9,1037798,"[8760867, 8760864, 3641634, 8760873, 4788864, ...","[[0.16464998, 0.24151766, 0.03492415, 0.126383...","[0.51637429, 0.44778393, -0.30746774, 0.142171...","[0.7704973263922338, 0.7411236886490814, 0.760..."


In [ ]:
# Qid, passage Id, cosine similarity

In [20]:
retrieved_docs_2 = pd.read_csv('relevant_docs_for_100_queries_test.csv')

In [21]:
retrieved_docs_2['docids'] = retrieved_docs_2['docids'].map(lambda vec: ast.literal_eval(','.join(re.sub(r'(?<=\d)(\s+)(?=-?\d)', ',', vec).splitlines())))

In [22]:
retrieved_docs['passage_ids'] = retrieved_docs_2['docids']

In [23]:
len(list(retrieved_docs_2['docids']))

43

In [24]:
retrieved_docs

,qid,docids,embeddings_docs,embeddings_queries,cosineDist,passage_ids
0,156493,"[8182161, 6139386, 3288600, 3288596, 2259183, ...","[[0.388196574, 0.244249168, 0.137481624, -0.00...","[0.31343375, 0.511575, -0.4668265, -0.1621775,...","[0.7706824998903609, 0.7534134139863174, 0.785...","[8182161, 6139386, 3288600, 3288596, 2259183, ..."
1,1110199,"[3838645, 554521, 398442, 4511137, 5218014, 81...","[[0.42657155, 0.19579138, 0.28220812, 0.102177...","[0.16202775, 0.795145, -0.46033125, -0.23275, ...","[0.7444827116295556, 0.7314614521048374, 0.769...","[3838645, 554521, 398442, 4511137, 5218014, 81..."
2,1063750,"[4788295, 2997653, 4337527, 6093907, 2981566, ...","[[0.355166434, 0.106690241, 0.14072722, -0.102...","[0.3308615, 0.711035, -0.1855625, -0.4082575, ...","[0.6862237295040958, 0.6637043734807618, 0.679...","[4788295, 2997653, 4337527, 6093907, 2981566, ..."
3,130510,"[1494936, 7125239, 799647, 7501563, 996732, 99...","[[0.377713008, -0.00231133514, -0.293942608, -...","[0.085094, 0.504474, -0.1749988, 0.2761902, 0....","[0.6697042269158479, 0.6340808852957971, 0.664...","[1494936, 7125239, 799647, 7501563, 996732, 99..."
4,489204,"[1479542, 1051498, 1950269, 2268657, 973923, 2...","[[0.550560748, 0.171315066, -0.165249143, -0.1...","[0.1761, 0.2345, -0.37276, 0.03199, 0.33063, 0...","[0.7521369989199707, 0.73420385978584, 0.76684...","[1479542, 1051498, 1950269, 2268657, 973923, 2..."
5,573724,"[2417985, 394136, 394139, 3365638, 7384325, 71...","[[0.26325938, 0.46548753, 0.02265388, 0.192594...","[0.51936, 0.6926975, -0.31155, -0.0028925, 0.2...","[0.7260829064947356, 0.7145796120067109, 0.734...","[2417985, 394136, 394139, 3365638, 7384325, 71..."
6,168216,"[3344828, 1381477, 3830857, 3174840, 1735360, ...","[[0.71077084, 0.12540752, 0.06708402, 0.216699...","[0.008874, 0.137065, -0.347945, 0.0185375, 0.2...","[0.6786883758466433, 0.6661525455599271, 0.694...","[3344828, 1381477, 3830857, 3174840, 1735360, ..."
7,1133167,"[5674622, 8160230, 8255705, 190809, 6467522, 8...","[[0.1223981, 0.49183291, -0.07783549, 0.525699...","[0.70916667, 0.31568333, -0.44523667, 0.076183...","[0.7666739141391231, 0.7425141883136608, 0.776...","[5674622, 8160230, 8255705, 190809, 6467522, 8..."
8,527433,"[8617271, 6186391, 269423, 1251475, 4903731, 1...","[[0.42529326, 0.29431083, -0.09624058, 0.23385...","[0.786148, 0.16785, -0.396498, -0.0035102, -0....","[0.6564955120524906, 0.6290717790738949, 0.663...","[8617271, 6186391, 269423, 1251475, 4903731, 1..."
9,1037798,"[8760867, 8760864, 3641634, 8760873, 4788864, ...","[[0.16464998, 0.24151766, 0.03492415, 0.126383...","[0.51637429, 0.44778393, -0.30746774, 0.142171...","[0.7704973263922338, 0.7411236886490814, 0.760...","[8760867, 8760864, 3641634, 8760873, 4788864, ..."


In [25]:
records = [{'qid': qid, 'pid': pid, 'cdist': sim} for qid in retrieved_docs['qid'] for pids, sims in zip(retrieved_docs['passage_ids'], retrieved_docs['cosineDist']) for pid, sim in zip(pids, sims)]

In [26]:
qids = []
pidsa = []
scores = []
for idx, row in retrieved_docs.iterrows():
    qid = row['qid']
    for index in range(0, len(row['passage_ids'])):
        if len(row['passage_ids']) == 1000 and len(row['cosineDist']) == 1000:
            qids.append(qid)
            pidsa.append(row['passage_ids'][index])
            scores.append(row['cosineDist'][index])

In [27]:
test_df = pd.DataFrame(data={'qid': qids, 'pid': pidsa, 'cdist': scores}, columns=['qid', 'pid', 'cdist'])

In [28]:
test_df.to_csv('final_df_test.csv', index=False)